In [3]:
# Large farm problem to test scalability of PIPS-NLP
# Yankai Cao and Victor M. Zavala
# University of Wisconsin-Madison, 2016

using JuMP
using Distributions
using Ipopt
using Plasmo

srand(123)
NS = 100;                   # number of scenarios
NP = 100;                   # number of products
S = collect(1:NS)           # scenario set
P = collect(1:NP)           # set of crops (1=wheat,2=corn,3=beets)

prcost = zeros(NP)
d = Uniform(100,200)
prcost = rand(d,NP)

pcost = zeros(NP)
d = Uniform(100,200)
pcost = rand(d,NP)

scost = zeros(NP)
scost = pcost - 50

demand = zeros(NP)
d = Uniform(100,300)
demand = rand(d,NP)/NP

# assign random data
yield = zeros(length(S),NP)
d = Uniform(5,20)
for j in 1:(NP-1)
    yield[S,j] = rand(d,1)[1]
end
d = Uniform(10,30)
yield[S,NP] = rand(d,NS)

sellub = zeros(NP)
d = Uniform(2000,8000)
sellub[P] = rand(d,NP)

# create PLASMO model and solve with PIPS-NLP
graph = GraphModel()
master = Model()
master_node = add_node(graph,master)

@variable(master, x[P] >= 0)
@variable(master, s2 >= 0)
@constraint(master, cap, (sum(x[j] for j in P) + s2) == 200)
@objective(master, Min, sum(prcost[j]*x[j] for j in P))
child_nodes = Array{Plasmo.NodeOrEdge}(NS)
for i in 1:NS
    bl = Model()
    child_node = add_node(graph,bl)
    child_nodes[i] = child_node
    @variable(bl, y[P] >= 0)    # crops purchase
    @variable(bl, 0<=w[j in P] <= sellub[j in P])    # crops sold
    @variable(bl, s[P] >= 0)
    @linkconstraint(graph, [j in P], yield[i,j]*master[:x][j]+y[j]-w[j] - s[j] == demand[j])
    @objective(bl, Min, 1.0/NS*sum(pcost[j]*y[j] - scost[j]*w[j] for j in P))
end
            
# solve with Ipopt
graph.solver = IpoptSolver()
solve(graph)

#println(getvalue(x))
#for i in 1:NS
#    println(getvalue(getindex(child_nodes[i],:w))) 
#end

Creating flattened graph model...
Finished model instantiation
This is Ipopt version 3.12.4, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:    40101
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    30101
                     variables with only lower bounds:    20101
                variables with lower and upper bounds:    10000
                     variables with only upper bounds:        0
Total number of equality constraints.................:    10001
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0300358e+02 1.